In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("orvile/traffic-and-weather-datasets")

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'

In [2]:
pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("orvile/traffic-and-weather-datasets")

print("Path to dataset files:", path)

Path to dataset files: /home/hero/.cache/kagglehub/datasets/orvile/traffic-and-weather-datasets/versions/1


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv("your_file.csv", parse_dates=['Date'])
df.sort_values('Date', inplace=True)

# Add useful columns
df['Daily Return %'] = df['Adj Close'].pct_change() * 100
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month Name'] = df['Date'].dt.month_name()
df['Weekday'] = df['Date'].dt.day_name()

# 1️⃣ Annual summary
annual_summary = df.groupby('Year').agg(
    Open=('Open', 'first'),
    Close=('Close', 'last'),
    High=('High', 'max'),
    Low=('Low', 'min'),
    Adj_Close=('Adj Close', 'last'),
    Total_Volume=('Volume', 'sum'),
    Avg_Daily_Return=('Daily Return %', 'mean'),
    Volatility=('Daily Return %', 'std')
).reset_index()

# 2️⃣ Monthly summary
monthly_summary = df.groupby(['Year', 'Month']).agg(
    Open=('Open', 'first'),
    Close=('Close', 'last'),
    High=('High', 'max'),
    Low=('Low', 'min'),
    Avg_Adj_Close=('Adj Close', 'mean'),
    Total_Volume=('Volume', 'sum'),
    Avg_Daily_Return=('Daily Return %', 'mean'),
    Volatility=('Daily Return %', 'std')
).reset_index()

# 3️⃣ Weekday pattern (across all years)
weekday_summary = df.groupby('Weekday').agg(
    Avg_Adj_Close=('Adj Close', 'mean'),
    Avg_Volume=('Volume', 'mean'),
    Avg_Daily_Return=('Daily Return %', 'mean'),
    Volatility=('Daily Return %', 'std')
).reindex(['Monday','Tuesday','Wednesday','Thursday','Friday']).reset_index()

# 4️⃣ Rolling 30-day volatility
df['Rolling_Volatility_30D'] = df['Daily Return %'].rolling(30).std()

# 5️⃣ Export detailed daily data (if needed in Power BI for custom visuals)
df.to_csv("detailed_daily_data.csv", index=False)

# 6️⃣ Export aggregated data
annual_summary.to_csv("annual_summary.csv", index=False)
monthly_summary.to_csv("monthly_summary.csv", index=False)
weekday_summary.to_csv("weekday_summary.csv", index=False)

# ---- Visuals for Power BI (optional) ----

# Price trend
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Adj Close'])
plt.title('Adjusted Close Price Over Time')
plt.xlabel('Date')
plt.ylabel('Adjusted Close Price')
plt.grid(True)
plt.savefig('adj_close_trend.png')
plt.close()

# Volume trend
plt.figure(figsize=(12, 6))
plt.bar(df['Date'], df['Volume'], width=1)
plt.title('Daily Trading Volume Over Time')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.savefig('volume_trend.png')
plt.close()

# Volatility trend
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Rolling_Volatility_30D'])
plt.title('30-Day Rolling Volatility (%)')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.grid(True)
plt.savefig('volatility_trend.png')
plt.close()

# Monthly heatmap
pivot = monthly_summary.pivot_table(index='Month', columns='Year', values='Avg_Adj_Close')
pivot = pivot.reindex(list(range(1,13)))
plt.figure(figsize=(14, 8))
sns.heatmap(pivot, annot=True, fmt=".1f", cmap='coolwarm')
plt.title('Monthly Avg Adjusted Close Heatmap')
plt.savefig('monthly_heatmap.png')
plt.close()

print("✅ All outputs generated: CSVs and plots ready for Power BI.")
